In [6]:
import pandas as pd
import re

In [7]:
import zipfile

with zipfile.ZipFile('customer_support_tickets.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')


In [8]:
import os
os.listdir('/content')


['.config',
 'customer_support_tickets.csv.zip',
 'customer_support_tickets.csv',
 'sample_data']

In [9]:
import pandas as pd

df = pd.read_csv('customer_support_tickets.csv')
df.head()


,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating
0,1,Marisa Obrien,carrollallison@example.com,32,Other,GoPro Hero,2021-03-22,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Social media,2023-06-01 12:15:36,NaN,NaN
1,2,Jessica Rios,clarkeashley@example.com,42,Female,LG Smart TV,2021-05-22,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Chat,2023-06-01 16:45:38,NaN,NaN
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Try capital clearly never color toward story.,Low,Social media,2023-06-01 07:29:40,2023-06-01 01:57:40,3.0
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,West decision evidence bit.,Low,Email,2023-06-01 00:12:42,2023-06-01 19:53:42,1.0


In [10]:
df.shape

(8469, 17)

In [11]:
df.columns

Index(['Ticket ID', 'Customer Name', 'Customer Email', 'Customer Age',
       'Customer Gender', 'Product Purchased', 'Date of Purchase',
       'Ticket Type', 'Ticket Subject', 'Ticket Description', 'Ticket Status',
       'Resolution', 'Ticket Priority', 'Ticket Channel',
       'First Response Time', 'Time to Resolution',
       'Customer Satisfaction Rating'],
      dtype='object')

In [12]:
df.isnull().sum()

,0
Ticket ID,0
Customer Name,0
Customer Email,0
Customer Age,0
Customer Gender,0
Product Purchased,0
Date of Purchase,0
Ticket Type,0
Ticket Subject,0
Ticket Description,0


In [13]:
df.head()

,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating
0,1,Marisa Obrien,carrollallison@example.com,32,Other,GoPro Hero,2021-03-22,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Social media,2023-06-01 12:15:36,NaN,NaN
1,2,Jessica Rios,clarkeashley@example.com,42,Female,LG Smart TV,2021-05-22,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Chat,2023-06-01 16:45:38,NaN,NaN
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Try capital clearly never color toward story.,Low,Social media,2023-06-01 07:29:40,2023-06-01 01:57:40,3.0
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,West decision evidence bit.,Low,Email,2023-06-01 00:12:42,2023-06-01 19:53:42,1.0


In [14]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-z\s]', ' ', text) # remove special characters
    text = re.sub(r'\s+', ' ', text) # remove extra spaces
    return text.strip()

df["clean_description"] = df["Ticket Description"].apply(clean_text)


In [15]:
def classify_issue(text):
    if any(word in text for word in ["payment", "billing", "charge"]):
        return "PAYMENT"
    elif any(word in text for word in ["login", "account", "password", "access"]):
        return "LOGIN"
    elif any(word in text for word in ["delivery", "shipping", "delay"]):
        return "DELIVERY"
    elif any(word in text for word in ["refund", "return"]):
        return "REFUND"
    elif any(word in text for word in ["bug", "error", "crash", "not working"]):
        return "BUG"
    else:
        return "GENERAL"

df["Issue_Category"] = df["clean_description"].apply(classify_issue)

In [16]:
def assign_priority(text):
    if any(word in text for word in ["not turning on", "data loss", "security"]):
        return "P0"
    elif any(word in text for word in ["payment", "refund", "account"]):
        return "P1"
    elif any(word in text for word in ["bug", "error", "issue"]):
        return "P2"
    else:
        return "P3"

df["Assigned_Priority"] = df["clean_description"].apply(assign_priority)


In [17]:
sla_map = {
    "P0": 2,
    "P1": 6,
    "P2": 24,
    "P3": 48
}

df["SLA_Hours"] = df["Assigned_Priority"].map(sla_map)

In [18]:
print("Total Tickets:", len(df))
print("\nTickets by Category:")
print(df["Issue_Category"].value_counts())

print("\nTickets by Priority:")
print(df["Assigned_Priority"].value_counts())

Total Tickets: 8469

Tickets by Category:
Issue_Category
GENERAL     5895
BUG         1062
LOGIN        763
PAYMENT      427
REFUND       224
DELIVERY      98
Name: count, dtype: int64

Tickets by Priority:
Assigned_Priority
P2    6334
P0     959
P1     878
P3     298
Name: count, dtype: int64


In [19]:
df.to_csv("final_ticket_triage_output.csv", index=False)